In [1]:
################################### setting path ###################################
import sys
sys.path.append('../')
sys.path.append('../../')
################################### import libs ###################################
from  pytorchyolo import  models_split_tiny
# from pytorchyolo import models
import numpy as np
import time
import torch
import math
import os
from split_framework.split_framework_dynamic import SplitFramework
import tqdm
import numpy as np
import requests, pickle
import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable
from terminaltables import AsciiTable
from pytorchyolo.utils.utils import load_classes, ap_per_class, get_batch_statistics, xywh2xyxy
from pytorchyolo.utils.datasets import ListDataset
from pytorchyolo.utils.transforms import DEFAULT_TRANSFORMS
from algorithm.manager_full import Manager
from split_framework.gnb import GNB
import random
from scipy.interpolate import griddata
import pandas as pd
from torchinfo import summary

import warnings

import torch.quantization

In [2]:
cfg_path = "../../pytorchyolo/config/yolov3-tiny.cfg"
model_path = "../ckpt/vidVRD.pth"

split_layer=8


In [3]:
model = models_split_tiny.load_model(cfg_path, model_path)
model.set_split_layer(split_layer) # layer <7
model = model.eval()
summary(model, input_size=(1, 3, 416, 416))

Layer (type:depth-idx)                   Output Shape              Param #
Darknet                                  [1, 256, 26, 26]          --
├─ModuleList: 1-1                        --                        8,459,262
│    └─Sequential: 2-1                   [1, 16, 416, 416]         --
│    │    └─Conv2d: 3-1                  [1, 16, 416, 416]         432
│    │    └─BatchNorm2d: 3-2             [1, 16, 416, 416]         32
│    │    └─LeakyReLU: 3-3               [1, 16, 416, 416]         --
│    └─Sequential: 2-2                   [1, 16, 208, 208]         --
│    │    └─MaxPool2d: 3-4               [1, 16, 208, 208]         --
│    └─Sequential: 2-3                   [1, 32, 208, 208]         --
│    │    └─Conv2d: 3-5                  [1, 32, 208, 208]         4,608
│    │    └─BatchNorm2d: 3-6             [1, 32, 208, 208]         64
│    │    └─LeakyReLU: 3-7               [1, 32, 208, 208]         --
│    └─Sequential: 2-4                   [1, 32, 104, 104]         --
│   

In [22]:
# model.half()

In [23]:
# summary(model, input_size=(1, 3, 416, 416), dtypes=[torch.float16])

In [28]:
model_int8 = torch.quantization.quantize_dynamic(
    model,  # your model
    {torch.nn.Linear},  # layers to quantize
    dtype=torch.qint8
)

In [32]:
x = torch.randn(1, 3, 416, 416)
x_q = torch.quantize_per_tensor(x, scale=0.1, zero_point=128, dtype=torch.quint8)

summary(model_int8, input_size=(1, 3, 416, 416))

Layer (type:depth-idx)                   Output Shape              Param #
Darknet                                  [1, 256, 26, 26]          --
├─ModuleList: 1-1                        --                        8,459,262
│    └─Sequential: 2-1                   [1, 16, 416, 416]         --
│    │    └─Conv2d: 3-1                  [1, 16, 416, 416]         432
│    │    └─BatchNorm2d: 3-2             [1, 16, 416, 416]         32
│    │    └─LeakyReLU: 3-3               [1, 16, 416, 416]         --
│    └─Sequential: 2-2                   [1, 16, 208, 208]         --
│    │    └─MaxPool2d: 3-4               [1, 16, 208, 208]         --
│    └─Sequential: 2-3                   [1, 32, 208, 208]         --
│    │    └─Conv2d: 3-5                  [1, 32, 208, 208]         4,608
│    │    └─BatchNorm2d: 3-6             [1, 32, 208, 208]         64
│    │    └─LeakyReLU: 3-7               [1, 32, 208, 208]         --
│    └─Sequential: 2-4                   [1, 32, 104, 104]         --
│   